# Dataset generators

| ! This notebook is deprecated, this API is not used throughout this repository. Will be removed. |
|-----------------------------------------|

______

In this notebook we show and develop generators for various datasets for testing.

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

import NegativeClassOptimization.config as config
import NegativeClassOptimization.datasets as datasets
import NegativeClassOptimization.utils as utils

In [2]:
df_global = pd.read_csv(config.DATA_SLACK_1_GLOBAL, sep='\t').iloc[:, 1:]

In [3]:
df_global.head(3)

,CDR3,Best,Slide,Energy,Structure,UID,Antigen
0,CARPENLLLLLWYFDVW,True,LLLLLWYFDVW,-112.82,137442-BRDSLLUDLS,3VRL_1873658_06a,3VRL
1,CARGLLLLLWYFDVW,True,LLLLLWYFDVW,-112.82,137442-BRDSLLUDLS,3VRL_7116990_04a,3VRL
2,CARGGLLLLLWYFDVW,True,LLLLLWYFDVW,-112.82,137442-BRDSLLUDLS,3VRL_7147788_05a,3VRL


In [4]:
df_global["Antigen"].unique()

array(['3VRL', '1NSN', '3RAJ', '5E94', '1H0D', '1WEJ', '1ADQ', '1FBI',
       '2YPV', '1OB1'], dtype=object)

## Random dataset

In [7]:
ds_random = datasets.RandomBindingDataset("random", 10000)
ds_random.df.head(3)

,CDR3,UID,Antigen,Dataset
0,RMIHWMMEIDCNGCAN,random_ebafcba8,random,random
1,NTLSRRWNYDFWHKHVE,random_00a021a2,random,random
2,QVKCYRHNIRD,random_01a6e919,random,random


## Positive dataset

In [8]:
antigen = "3VRL"
df_ag = df_global.loc[df_global["Antigen"] == antigen]
ds_pos = datasets.PositiveBindingDataset(f"positive_{antigen}", df=df_ag, antigen=antigen)
ds_pos.df.head(3)

,ID_slide_Variant,CDR3,Best,Slide,Energy,Structure,UID,Antigen,Dataset
0,1873658_06a,CARPENLLLLLWYFDVW,True,LLLLLWYFDVW,-112.82,137442-BRDSLLUDLS,3VRL_1873658_06a,3VRL,positive_3VRL
1,7116990_04a,CARGLLLLLWYFDVW,True,LLLLLWYFDVW,-112.82,137442-BRDSLLUDLS,3VRL_7116990_04a,3VRL,positive_3VRL
2,7147788_05a,CARGGLLLLLWYFDVW,True,LLLLLWYFDVW,-112.82,137442-BRDSLLUDLS,3VRL_7147788_05a,3VRL,positive_3VRL


## Negative dataset

In [9]:
antigens_neg = ['1NSN', '3RAJ', '5E94']
positive_datasets_for_neg = []
for antigen in antigens_neg:
    df_ag = df_global.loc[df_global["Antigen"] == antigen]
    ds_pos = datasets.PositiveBindingDataset(f"positive_{antigen}", df=df_ag, antigen=antigen)
    positive_datasets_for_neg.append(ds_pos)

ds_neg = datasets.NegativeBindingDataset(
    name="negative",
    positive_datasets=positive_datasets_for_neg,
    random_datasets=[ds_random]
)

In [10]:
ds_neg.df.head(3)

,CDR3,UID,Antigen,Dataset
70000,CVLQLRLPFAWFAYW,1NSN_5278578_01a,1NSN,positive_1NSN
70001,CELLLWLREDYFDYW,1NSN_711507_01a,1NSN,positive_1NSN
70002,CARSDILLWLPEEIFFAYW,1NSN_7211476_04a,1NSN,positive_1NSN


## CompleteDataset

In [11]:
ds = datasets.CompleteBindingDataset(
    "complete",
    positive_dataset=ds_pos,
    negative_dataset=ds_neg,
)

In [12]:
ds.df.head(3)

,ID_slide_Variant,CDR3,Best,Slide,Energy,Structure,UID,Antigen,Dataset,binder
210000,825477_03a,CARIFLLLRYLWYFDVW,True,IFLLLRYLWYF,-110.91,149279-SUDDULSUSL,5E94_825477_03a,5E94,positive_5E94,True
210001,1565352_06a,CARQGVFFLLLRFWFAYW,True,FFLLLRFWFAY,-110.75,149279-SUDDULSUSL,5E94_1565352_06a,5E94,positive_5E94,True
210002,4081140_01a,CFYLLLRLYWYFDVW,True,FYLLLRLYWYF,-109.99,149279-SUDDULSUSL,5E94_4081140_01a,5E94,positive_5E94,True


## Building a `CompleteDataset` conveniently

In [14]:
data_source_global = config.DATA_SLACK_1_GLOBAL

antigen_pos = "3VRL"
subsample_pos_size = None

antigens_neg = ['1NSN', '3RAJ', '5E94']
subsample_neg_size_per_ag = None

num_random = 70000

In [ ]:
df_global = pd.read_csv(data_source_global, sep='\t')

## Pos
df_ag = df_global.loc[df_global["Antigen"] == antigen_pos]
ds_pos = datasets.PositiveBindingDataset(f"positive_{antigen_pos}", df=df_ag, antigen=antigen_pos)

## Neg
ds_random = datasets.RandomBindingDataset("random", num_random)

positive_datasets_for_neg = []
for antigen_neg in antigens_neg:
    df_ag = df_global.loc[df_global["Antigen"] == antigen_neg]
    ds_pos = datasets.PositiveBindingDataset(f"positive_{antigen_neg}", df=df_ag, antigen=antigen_neg)
    positive_datasets_for_neg.append(ds_pos)

ds_neg = datasets.NegativeBindingDataset(
    name="negative",
    positive_datasets=positive_datasets_for_neg,
    random_datasets=[ds_random]
)

## Complete
ds = datasets.CompleteBindingDataset(
    "complete",
    positive_dataset=ds_pos,
    negative_dataset=ds_neg,
)

In [ ]:
metadata = {
    ""
}
# ds.record_dataset(fp, metadata)